Inspired by: https://docs.databricks.com/_static/notebooks/mlflow/mlflow-quick-start-training.html

# TODO
- What to log
    - model name
    - model hyper-prameters
    - model features
    - performances
- Model
    - Save
    - Load
- Create experiments
- Search run for a given experiment with SQL query - https://docs.faculty.ai/user-guide/experiments/index.html#experiments-multiple
- Create runs
- Shared DB for mlflow -> check https://github.com/dmatrix/mlflow-tests/tree/master/py/mlflow/server


# Doc

## MLflow Tracking
documentation > https://www.mlflow.org/docs/latest/tracking.html

- Can be used to to track experiments to record and compare parameters and results.

**Vocabulary**
- *run*: corresponds to a single execution of model training code. Each run can record different informations (model parameters, metrics, tags, artifacts, etc).
- *experiment*: the primary unit of organization and access control for MLflow runs; all MLflow runs belong to an experiment. Experiments let you visualize, search for, and compare runs, as well as download run artifacts and metadata for analysis in other tools.
- *MLflow entities*: runs, parameters, metrics, tags, notes, metadata, etc
- ...

**What can be recorded by an MLflow run?** > https://www.mlflow.org/docs/latest/tracking.html#concepts

**Where runs are recorded?** > https://www.mlflow.org/docs/latest/tracking.html#where-runs-are-recorded

They can be recorded
- to local files (by default to *mlruns* directory)
    - `mlflow ui`
- to SQLAlchemy compatible database
    - `mlflow.set_tracking_uri('sqlite:///mlflow.db')`
    - `mlflow ui --backend-store-uri sqlite:///mlflow.db`
- remotely to a tracking server

To show the current tracking uri `mlflow.get_tracking_uri()`
    
**How they are recorded** > https://www.mlflow.org/docs/latest/tracking.html#how-runs-and-artifacts-are-recorded

MLflow uses two components for storage:
- backend store: for MLflow entities (runs, parameters, metrics, tags, notes, metadata, etc)
- artifact store: for artifacts (files, models, images, in-memory objects, or model summary, etc)

**How to vizualise the logged runs?**
- You can use the MLflow tracking ui `mlflow ui` (should be run from the folder where the *mlruns* directory is located)

### Logging

**What to log**


**How**
- Manual logging > https://www.mlflow.org/docs/latest/tracking.html#logging-functions
    - Log the fitted model: `mlflow.sklearn.log_model(rf, 'random-forest-model')`
    - Log the model parameters:
        - One parameter at a time: `mlflow.log_param('num_trees', n_estimators)`
        - A dict of parameters: `mlflow.log_parms({'num_trees', n_estimators, 'alpha', 0.04})`
    - Log the evaluation metrics: `mlflow.log_metric('mse', mse)`
    - Log other artifacts: `mlflow.log_artifact('predictions.csv')`

- Automatic logging with MLflow autolog
    - With MLflow's autologging capabilities, a single line of code automatically logs the resulting model, the parameters used to create the model, and a model score > https://www.mlflow.org/docs/latest/tracking.html#automatic-logging
    - Call `mlflow.<framework>.autolog()` API before running training code to log model-specific metrics, parameters, and model artifacts. Supports many ML frameworks (sklearn, tensorflow, etc).

### Experiments
**How to set an experiment?**

- with 

In [26]:
import numpy as np
import pandas as pd
import mlflow

from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

/Users/alaa.bakhti/miniconda3/envs/dsp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Step 1: basic pipeline

- Load dataset

In [21]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

def get_dataset() -> pd.DataFrame:
    db = load_diabetes()
    X, y = db.data, db.target
    return train_test_split(X, y, random_state=42)

X_train, X_test, y_train, y_test = get_dataset()
X_train.shape, X_test.shape

((331, 10), (111, 10))

- Train model

In [23]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R2 = {r2:.2f}')
    return rmse, mae, r2

def train_model(X_train, X_test, y_train, y_test: pd.DataFrame, model_class, **model_kwargs) -> int:
    model = model_class(**model_kwargs)
    model.fit(X_train, y_train)
    evaluate_model(model, X_test, y_test)

In [24]:
model_kwargs = {'alpha': 0.01, 'l1_ratio': 0.75}
train_model(X_train, X_test, y_train, y_test, ElasticNet, **model_kwargs)

/Users/alaa.bakhti/miniconda3/envs/dsp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RMSE = 55.11, MAE = 45.22, R2 = 0.45


In [25]:
model_kwargs = {'alpha': 0.04, 'l1_ratio': 0.5}
train_model(X_train, X_test, y_train, y_test, ElasticNet, **model_kwargs)

/Users/alaa.bakhti/miniconda3/envs/dsp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RMSE = 66.21, MAE = 57.36, R2 = 0.21


# Step 2:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mlflow.log_metric('rmse', rmse)
    mlflow.log_metric('mae', mae)
    mlflow.log_metric('r2', r2)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R2 = {r2:.2f}')
    return rmse, mae, r2

def train_model(X_train, X_test, y_train, y_test: pd.DataFrame, model_class, **model_kwargs) -> int:
    model = model_class(**model_kwargs)
    model.fit(X_train, y_train)
    for hyper_parameter, value in model_kwargs:
        mlflow.log_param(hyper_parameter, value)
    # Or better: mlflow.log_params(model_kwargs)
    mlflow.sklearn.log_model(model, "model")
    evaluate_model(model, X_test, y_test)

In [27]:
hyper_parameters_list = [
    {'alpha': 0.01, 'l1_ratio': 0.75},
    {'alpha': 0.04, 'l1_ratio': 0.5}
]

for model_hyper_parameters_dict in hyper_parameters_list:
    train_model(X_train, X_test, y_train, y_test, ElasticNet, **model_hyper_parameters_dict)

/Users/alaa.bakhti/miniconda3/envs/dsp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RMSE = 55.11, MAE = 45.22, R2 = 0.45
RMSE = 66.21, MAE = 57.36, R2 = 0.21


In [6]:
model_dict_list = [
    {'model_class' : ElasticNet, 'model_kwargs': {'alpha': 0.01, 'l1_ratio': 0.75}},
    {'model_class' : RandomForestRegressor, 'model_kwargs': {'n_estimators': 100, 'max_depth': 6, 'max_features': 3}}
]
# train_model(X_train, X_test, y_train, y_test, model_dict['model_class'], **model_dict['model_kwargs'])

# Mlflow

In [17]:
#mlflow_backend_store_sqlite_db_uri = 'sqlite:////tmp/mlruns.db'
#mlflow.set_tracking_uri(mlflow_backend_store_sqlite_db_uri)

"""
Using sqlite:///<path>/mlruns.db for backend store and /tmp/mlruns for artifact store
Run mlflow server with:
mlflow server --backend-store-uri sqlite:////tmp/mlruns.db --default-artifact-root /tmp/mlruns
"""
mlflow.set_tracking_uri('http://127.0.0.1:5000')

/Users/alaa.bakhti/miniconda3/envs/dsp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
mlflow.sklearn.autolog()

#mlflow.set_experiment('my_experiment 1')

for i in range(5):
    for model_dict in model_dict_list:
        with mlflow.start_run():
            train_model(X_train, X_test, y_train, y_test, model_dict['model_class'], **model_dict['model_kwargs'])

/Users/alaa.bakhti/miniconda3/envs/dsp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RMSE = 55.11, MAE = 45.22, R2 = 0.45
RMSE = 52.95, MAE = 42.83, R2 = 0.49
RMSE = 55.11, MAE = 45.22, R2 = 0.45
RMSE = 52.51, MAE = 42.41, R2 = 0.50
RMSE = 55.11, MAE = 45.22, R2 = 0.45
RMSE = 53.59, MAE = 42.52, R2 = 0.48
RMSE = 55.11, MAE = 45.22, R2 = 0.45
RMSE = 52.71, MAE = 42.39, R2 = 0.50
RMSE = 55.11, MAE = 45.22, R2 = 0.45
RMSE = 52.67, MAE = 42.10, R2 = 0.50


In [20]:
!ls

mflow.ipynb      mlflow-old.ipynb mlflow.db        mlruns


/Users/alaa.bakhti/miniconda3/envs/dsp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
mlflow.search_runs(filter_string="metric.training_mae < 30")

/Users/alaa.bakhti/miniconda3/envs/dsp/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_score,metrics.training_mae,metrics.training_r2_score,metrics.training_mse,...,params.min_impurity_decrease,params.warm_start,params.max_samples,params.criterion,tags.estimator_class,tags.mlflow.user,tags.mlflow.log-model.history,tags.mlflow.source.type,tags.estimator_name,tags.mlflow.source.name
0,641d9f9708b14bc69f3d3776d7c230a3,2,FINISHED,./mlruns/2/641d9f9708b14bc69f3d3776d7c230a3/ar...,2021-05-20 12:43:38.187000+00:00,2021-05-20 12:43:38.850000+00:00,0.790369,29.658545,0.790369,1260.867872,...,0.0,False,None,mse,sklearn.ensemble._forest.RandomForestRegressor,alaa.bakhti,"[{""run_id"": ""641d9f9708b14bc69f3d3776d7c230a3""...",LOCAL,RandomForestRegressor,/Users/alaa.bakhti/miniconda3/envs/dsp/lib/pyt...
1,67cf7cf4c4a34b618c566b0d951bf094,2,FINISHED,./mlruns/2/67cf7cf4c4a34b618c566b0d951bf094/ar...,2021-05-12 23:54:31.977000+00:00,2021-05-12 23:54:32.474000+00:00,0.781430,29.775595,0.781430,1270.739478,...,0.0,False,None,mse,sklearn.ensemble._forest.RandomForestRegressor,alaa.bakhti,"[{""run_id"": ""67cf7cf4c4a34b618c566b0d951bf094""...",LOCAL,RandomForestRegressor,/Users/alaa.bakhti/miniconda3/envs/dsp/lib/pyt...
